In [1]:
%cd C:\Users\yukir\Documents\GitHub\Text_Mining\py

from nlp_processing import *
set_global_determinism(seed=1337)

%cd C:\Users\yukir\Documents\Monicas_workspace\Derma\FINAL_DERMA\TF_IDF(LDA)

C:\Users\yukir\Documents\GitHub\Text_Mining\py
True
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7620730895153757517
]
tf 2.5.3
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라
set_global_determinism(seed=1337) 이거 꼭 해라


c:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\scipy\io\matlab\mio5.py:98: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from .mio5_utils import VarReader5


C:\Users\yukir\Documents\Monicas_workspace\Derma\FINAL_DERMA\TF_IDF(LDA)


In [2]:
import openpyxl
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from gensim import corpora, models
from gensim.models import CoherenceModel
import gensim
import matplotlib.pyplot as plt

from gensim.models.ldamodel import LdaModel 
from gensim.models.callbacks import CoherenceMetric 
from gensim import corpora 
from gensim.models.callbacks import PerplexityMetric 
import logging 
import pickle 
import pyLDAvis

from gensim.models.coherencemodel import CoherenceModel 
import matplotlib.pyplot as plt

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from konlpy.tag import Mecab 
from konlpy.tag import Okt
okt = Okt()
from tqdm import tqdm 
import re 
import pickle 
import csv

### 띄어쓰기

In [3]:
reviews_df = pd.read_excel("..//data//[0530]for_keywords_d.xlsx", index_col=0)
reviews_df.head()


,hospital,star,reviews
0,오라클피부과의원 청주북문로점,6.8,저랑은 안맞는 거같아요
1,오라클피부과의원 청주북문로점,6.8,의사선생님은 어떻게든 이유를 찾아주시려고 하는 거같은데
2,오라클피부과의원 청주북문로점,6.8,아무런
3,오라클피부과의원 청주북문로점,6.8,효과나 뭐없었어요
4,오라클피부과의원 청주북문로점,6.8,두리뭉실한


In [4]:
print(reviews_df.shape)
reviews_df = reviews_df.drop_duplicates()
reviews_df = reviews_df.dropna()
print(reviews_df.shape)

(226258, 3)
(226258, 3)


In [7]:
reviews_df

,hospital,star,reviews
0,오라클피부과의원 청주북문로점,6.8,저랑은 안맞는 거같아요
1,오라클피부과의원 청주북문로점,6.8,의사선생님은 어떻게든 이유를 찾아주시려고 하는 거같은데
2,오라클피부과의원 청주북문로점,6.8,아무런
3,오라클피부과의원 청주북문로점,6.8,효과나 뭐없었어요
4,오라클피부과의원 청주북문로점,6.8,두리뭉실한
...,...,...,...
234572,압구정톡스앤필의원,8.5,병원은 소쏘입니다
234573,은평미피부과의원,5,불광역6번출 구쪽에 위치한 은평미
234574,은평미피부과의원,5,피부과를 방문하였습니다2 5층에 위치하고 있으며
234575,은평미피부과의원,5,입간 판이따로 없어서


In [5]:
data = reviews_df.copy()
# data = data[20000:]
print(data.shape)

(226258, 3)


In [8]:
len(data.hospital.unique())

2173

In [9]:
data = data[['reviews']]

In [10]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

In [11]:
def stemming (text):
    morphs_list = []
    one_words = []
    result = []

    pos = okt.pos(text, join = True)
    # print(pos)
    for j in pos:
        if j.split('/')[1] == 'Noun':
            j = j.split('/')[0]
            morphs_list.append(j)
                    
        elif j.split('/')[1] =='Adjective':
            k = okt.morphs(j,  stem= True)
            k = k[0]
            morphs_list.append(k)
        elif j.split('/')[1] =='Verb':
            v = okt.morphs(j,  stem= True)
            v = v[0]
            morphs_list.append(v)

    for i in morphs_list:
        if len(i) != 1:
            one_words.append(i)
    

    for i in one_words:
        if i not in stopwords:
            result.append(i)

    return result

In [13]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|a-zA-Z]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 
    
def get_nouns(tokenizer, sentence): 
    """ 단어의 길이가 2이상인 일반명사(NNG),
     고유명사(NNP), 외국어(SL)만을 반환한다. 
     :param tokenizer: 
     :param sentence: :return: """ 
    tagged = tokenizer.nouns(sentence)
    nouns = [s for s in tagged if len(s)>1] 
     
    return nouns 

    
def save_processed_data(processed_data): 
    """ 토큰 분리한 데이터를 csv로 저장 :param processed_data: :return: """ 
    
    with open('lsa_token_pos.csv', 'w', newline='', encoding='utf-8') as f: 
        writer = csv.writer(f) 
        for data in processed_data:
             writer.writerow(data)

In [14]:
review_list = data.reviews.tolist()

In [15]:
from hanspell import spell_checker

In [16]:
data

,reviews
0,저랑은 안맞는 거같아요
1,의사선생님은 어떻게든 이유를 찾아주시려고 하는 거같은데
2,아무런
3,효과나 뭐없었어요
4,두리뭉실한
...,...
234572,병원은 소쏘입니다
234573,불광역6번출 구쪽에 위치한 은평미
234574,피부과를 방문하였습니다2 5층에 위치하고 있으며
234575,입간 판이따로 없어서


### TF_IDF

In [17]:
print(len(data))
data = data.dropna()
print(len(data))
data = data.drop_duplicates('reviews')
print(len(data))

226258
226258
183653


In [18]:
for sent in tqdm(data['reviews']):
    try:
        sentence = clean_text(sent.replace('\n', '').strip())
        # print(sentence)
    except:
        # print(sent)
        pass

100%|██████████| 183653/183653 [00:00<00:00, 492325.25it/s]


In [19]:
def tokenize(df): 
    tokenizer = okt
    processed_data = [] 
    for sent in tqdm(df['reviews']):
        sentence = clean_text(sent.replace('\n', '').strip()) 
        processed_data.append(get_nouns(tokenizer, sentence)) 
        
    return processed_data

In [20]:
## 오래걸리니까 조심

if __name__ == '__main__':
     # df = pd.read_csv('negative.csv', index_col = 0)
     processed_data = tokenize(data) # description 부분을 토크나이징 한다. 
     
     # 토큰 분리한 데이터를 저장 
     save_processed_data(processed_data)


100%|██████████| 183653/183653 [12:59<00:00, 235.62it/s]


In [21]:
detokenized_doc = []
for i in range(len(processed_data)):
    t = ' '.join(processed_data[i])
    detokenized_doc.append(t)

data['detokenized_doc'] = detokenized_doc # 다시 text['headline_text']에 재저장

data['detokenized_doc'][:5]

0             
1    의사 선생님 이유
2             
3           효과
4           두리
Name: detokenized_doc, dtype: object

In [22]:
from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words=stopwords_kor,             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[가-힣0-9]{1,}',  # num chars > 3
                             max_features=100,             # max number of uniq words
                            ) 
data_vectorized = vectorizer.fit_transform(data['detokenized_doc'])
data_vectorized.shape # TF-IDF 행렬의 크기 확인

(183653, 100)

In [24]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
dist = np.sum(data_vectorized, axis=0)
    
df_freq = pd.DataFrame(dist, columns=vocab)
print(df_freq.shape)

100
(1, 100)


In [25]:
df_freq.T.sort_values(by=0, ascending=False).head(100)
df_freq_T = df_freq.T.reset_index()
df_freq_T.columns = ["words", "freq"]
df_freq_T.tail()

,words,freq
95,화상,619.708634
96,확인,533.144443
97,환자,2289.847218
98,효과,3590.014215
99,흉터,708.694858


In [26]:
print(df_freq_T.shape)
df_use = df_freq_T.drop_duplicates()
print(df_use.shape)

(100, 2)
(100, 2)


In [27]:
df_use = df_use.sort_values(by="freq", ascending=False)
df_use.head(100)

,words,freq
41,설명,6648.391932
46,시술,5138.830105
75,직원,5136.886294
79,처방,5101.869054
55,여드름,3981.246191
...,...,...
26,목적,460.088051
8,공장,456.578905
74,지인,444.985210
22,링크,431.784375


In [28]:
df_use.to_excel('[0530]TF_IDF_splited.xlsx')

### **LDA**

In [54]:
from nltk.corpus import stopwords  
stopwords.words('korean')[:10] 

stopwords_kor = stopwords.words('korean')

In [55]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(analyzer='word',       
#                              min_df=10,                        # minimum reqd occurences of a word 
#                              stop_words=stopwords_kor,             # remove stop words
#                              lowercase=True,                   # convert all words to lowercase
#                              token_pattern='[가-힣0-9]{1,}',  # num chars > 3
#                              max_features=1000,             # max number of uniq words
#                             ) 
# data_vectorized = vectorizer.fit_transform(data['detokenized_doc'])
# data_vectorized.shape # TF-IDF 행렬의 크기 확인

In [29]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  0.9556500574452907 %


In [30]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=5,               # Number of topics
                                      max_iter=20,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=1337,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_top=lda_model.fit_transform(data_vectorized)

print(lda_model.components_)
print(lda_model.components_.shape) 

[[2.00920331e-01 2.00566384e-01 2.00892014e-01 2.00808081e-01
  2.00254615e-01 2.00831383e-01 2.00636226e-01 2.00897730e-01
  4.49633016e+02 2.00853273e-01 2.00933301e-01 2.00763644e-01
  2.00349252e-01 2.00519528e-01 2.00601157e-01 2.00370059e-01
  2.20824839e+03 2.00565540e-01 1.16944092e+03 2.00539436e-01
  2.00831434e-01 2.02290859e-01 2.00601451e-01 2.00806272e-01
  2.00794089e-01 2.00810321e-01 2.02458192e-01 2.00618855e-01
  8.47208596e+02 2.01277150e-01 6.05500265e+02 2.00585601e-01
  2.00774876e-01 2.00798756e-01 2.00374435e-01 2.00563483e-01
  2.01500200e-01 2.00760955e-01 2.02170946e-01 2.00669742e-01
  2.01064122e-01 6.68763684e+03 2.00402593e-01 2.01428651e-01
  2.00778881e-01 2.00411113e-01 2.01181587e-01 2.00519141e-01
  2.00637653e-01 2.00891701e-01 2.00962198e-01 2.01487252e-01
  2.00736506e-01 2.00828614e-01 2.00967733e-01 2.01352181e-01
  2.01951772e-01 2.01604870e-01 2.00484391e-01 2.00788977e-01
  8.27188382e+02 2.00650323e-01 2.00373785e-01 2.00997523e-01
  2.0061

In [71]:
# Create Document - Topic Matrix
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(data['reviews'].tolist()))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,dominant_topic
Doc0,0.100000,0.100000,0.100000,0.590000,0.100000,3
Doc1,0.070000,0.330000,0.460000,0.070000,0.070000,2
Doc2,0.070000,0.070000,0.070000,0.720000,0.070000,3
Doc3,0.100000,0.600000,0.100000,0.100000,0.100000,1
Doc4,0.510000,0.260000,0.060000,0.060000,0.120000,0
Doc5,0.050000,0.170000,0.050000,0.580000,0.140000,3
Doc6,0.080000,0.080000,0.080000,0.690000,0.080000,3
Doc7,0.690000,0.080000,0.080000,0.080000,0.080000,0
Doc8,0.290000,0.070000,0.070000,0.070000,0.500000,4
Doc9,0.430000,0.070000,0.060000,0.380000,0.060000,0


In [72]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
get_topics(lda_model.components_,terms)

Topic 1: [('건물', 492.98), ('동네', 432.0), ('습진', 427.53), ('주차', 394.11), ('걱정', 357.55), ('흉터', 335.19), ('가게', 332.89), ('설명', 328.8), ('처치', 324.37), ('이용', 317.13)]
Topic 2: [('설명', 632.11), ('직원', 602.28), ('시설', 553.91), ('대기', 492.93), ('아토피', 474.89), ('원장님', 439.65), ('접수', 432.22), ('환자', 416.19), ('질문', 375.11), ('친절', 370.21)]
Topic 3: [('처방', 1407.57), ('알레르기', 968.47), ('피부질환', 913.54), ('연고', 825.63), ('증상', 814.1), ('약물', 541.36), ('피부염', 524.0), ('설명', 489.99), ('원인', 480.05), ('약도', 357.89)]
Topic 4: [('시술', 1662.91), ('제거', 1201.05), ('보톡스', 1110.98), ('레이저', 1101.98), ('상담', 1033.5), ('가격', 791.86), ('사마귀', 774.89), ('효과', 669.61), ('제모', 547.72), ('예약', 544.28)]
Topic 5: [('여드름', 1687.81), ('주사', 1530.97), ('염증', 1302.67), ('압출', 663.39), ('무좀', 520.41), ('관리', 517.73), ('피부관리', 458.18), ('진단', 425.0), ('트러블', 378.04), ('설명', 360.68)]


In [73]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

c:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\pyLDAvis\_prepare.py:228: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info  = pd.DataFrame({'saliency': saliency, 'Term': vocab, \
c:\Users\yukir\anaconda3\envs\NLP\lib\site-packages\sklearn\manifold\_t_sne.py:691: FutureWarning: 'square_distances' has been introduced in 0.24 to help phase out legacy squaring behavior. The 'legacy' setting will be removed in 1.1 (renaming of 0.26), and the default setting will be changed to True. In 1.3, 'square_distances' will be removed altogether, and distances will be squared by default. Set 'square_distances'=True to silence this warning.
  warnings.warn(


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3      150.222153  -72.269554       1        1  26.077613
4       51.163364 -159.968018       2        1  21.607652
2      -73.144554   21.523815       3        1  18.277393
1      -66.410133 -110.606438       4        1  17.186782
0       54.417324   12.443444       5        1  16.850559, topic_info=    Term         Freq        Total Category  logprob  loglift
53   여드름  1678.000000  1678.000000  Default  30.0000  30.0000
71    주사  1716.000000  1716.000000  Default  29.0000  29.0000
55    염증  1295.000000  1295.000000  Default  28.0000  28.0000
47  알레르기  1001.000000  1001.000000  Default  27.0000  27.0000
43    시술  1640.000000  1640.000000  Default  26.0000  26.0000
..   ...          ...          ...      ...      ...      ...
75    직원   250.442642  1703.323275   Topic5  -3.7471  -0.1363
16    대기   228.516527  1562.243066   Topic5  -3.8387  -0.1415
15    느낌   156.157201   928.609619   Topic5  -4.2195  -0.0020
79    처방   192.976525  2284.578986   Topic5  -4.0078  -0.6906
97    환자   148.361354   997.654542   Topic5  -4.2707  -0.1250

[204 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         5  0.997680   가게
1         1  0.706728   가격
1         2  0.237918   가격
1         5  0.054209   가격
2         1  0.110624   개선
...     ...       ...  ...
98        3  0.185603   효과
98        4  0.088530   효과
98        5  0.046595   효과
99        1  0.002731   흉터
99        5  0.996938   흉터

[217 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 5, 3, 2, 1])